# Funciones útiles

In [25]:
def Extraer_Ultimos_Numeros(df, Columna, Nombre_Nueva_Columna):
    
    """
    Esta función toma un DataFrame, una columna y un nombre para una nueva columna como entrada.
    Extrae los últimos números de cada cadena en la columna dada del DataFrame.
    Luego, si hay comas en los números extraídos, las convierte en puntos.
    Convierte la columna resultante en tipo float y la agrega al DataFrame con el nombre especificado.
    Devuelve el DataFrame modificado.
    
    Parámetros:
    - df: DataFrame - El DataFrame al que se aplicará la operación.
    - columna: str - El nombre de la columna del DataFrame de donde se extraerán los números.
    - nueva_columna_nombre: str - El nombre que se asignará a la nueva columna creada con los números extraídos.
    
    Retorna:
    - DataFrame: El DataFrame modificado con la nueva columna.
    """
    
    # Inicializar una lista para almacenar todas las listas de números extraídos
    Listas_Numeros = []
    
    # Iterar sobre cada fila de la columna especificada
    for i in df[Columna]:
        # Encontrar todos los números en la fila y convertirlos a una lista
        Numeros = re.findall(r'[\d,]+', i)
        # Convertir la lista de números en una lista de floats
        Numeros = [float(j.replace(',', '.')) for j in Numeros]
        # Agregar la lista de números a la lista de listas
        Listas_Numeros.append(Numeros)
    
    # Tomar el último elemento de cada lista, reemplazar comas por puntos, convertir a float y agregar al DataFrame
    df[Nombre_Nueva_Columna] = [Lista[-1] for Lista in Listas_Numeros]
    
    return df

In [8]:
def Comparar_Df_y_Copiar(df1, df2, ColumnaA, ColumnaB, ColumnaC, ColumnaD):
    
    """
    Compara los valores de la columna ColumnaA del DataFrame df1 con los valores de la columna ColumnaB del DataFrame df2.
    Si hay coincidencias, copia los valores correspondientes de la columna ColumnaD de df2 a la columna ColumnaC de df1.

    Parámetros:
        df1 (DataFrame): El DataFrame de origen donde se realizarán las comparaciones y copias.
        df2 (DataFrame): El DataFrame que se utilizará como referencia para las comparaciones.
        ColumnaA (str): El nombre de la columna en df1 que se utilizará para la comparación.
        ColumnaB (str): El nombre de la columna en df2 que se utilizará para la comparación.
        ColumnaC (str): El nombre de la columna en df1 donde se copiarán los valores de df2.
        ColumnaD (str): El nombre de la columna en df2 de donde se tomarán los valores a copiar.

    Retorna:
        DataFrame: El DataFrame df1 con los valores actualizados en la columna ColumnaC donde hubo coincidencias.
    """

    # Renombrar temporalmente las columnas si tienen el mismo nombre
    if ColumnaA == ColumnaB:
        df2 = df2.rename(columns={ColumnaB: f"{ColumnaB}_temp"})
        ColumnaB = f"{ColumnaB}_temp"
    if ColumnaC == ColumnaD:
        df2 = df2.rename(columns={ColumnaD: f"{ColumnaD}_temp"})
        ColumnaD = f"{ColumnaD}_temp"
    
    # Realizar un merge en las columnas ColumnaA y ColumnaB para identificar coincidencias
    merged_df = pd.merge(df1, df2[[ColumnaB, ColumnaD]], left_on=ColumnaA, right_on=ColumnaB, how='left', suffixes=('', '_df2'))

    # Copiar los valores de ColumnaD_df2 a ColumnaC donde hay coincidencias
    merged_df[ColumnaC] = merged_df.apply(lambda x: x[ColumnaD] if pd.notnull(x[ColumnaD]) else x[ColumnaC], axis=1)

    # Eliminar las columnas adicionales generadas por el merge
    merged_df.drop(columns=[ColumnaB, ColumnaD], inplace=True)

    # Restaurar los nombres originales de las columnas si se han renombrado temporalmente
    if '_temp' in ColumnaB:
        merged_df = merged_df.rename(columns={ColumnaB: ColumnaA})
    if '_temp' in ColumnaD:
        merged_df = merged_df.rename(columns={ColumnaD: ColumnaC})

    return merged_df

In [27]:
def Nueva(df1, df2, columna_a, columna_b, columna_c, columna_d):
    """
    Compara los valores de la columna 'columna_a' del DataFrame 'df1' con los valores de la columna 'columna_b' del DataFrame 'df2'.
    Si hay coincidencias exactas, copia los valores correspondientes de la columna 'columna_d' de 'df2' a la columna 'columna_c' de 'df1'.

    Args:
        df1 (DataFrame): El DataFrame de origen donde se realizarán las comparaciones y copias.
        df2 (DataFrame): El DataFrame que se utilizará como referencia para las comparaciones.
        columna_a (str): El nombre de la columna en 'df1' que se utilizará para la comparación.
        columna_b (str): El nombre de la columna en 'df2' que se utilizará para la comparación.
        columna_c (str): El nombre de la columna en 'df1' donde se copiarán los valores de 'df2'.
        columna_d (str): El nombre de la columna en 'df2' de donde se tomarán los valores a copiar.

    Returns:
        DataFrame: El DataFrame 'df1' con los valores actualizados en la columna 'columna_c' donde hubo coincidencias exactas.
    """
    # Filtra los valores de df1 que coinciden exactamente en columna_a con los de columna_b en df2
    valores_coincidentes = set(df1[columna_a]).intersection(df2[columna_b])

    # Actualiza los valores en columna_c de df1 con los valores de columna_d de df2 donde hay coincidencias exactas
    for valor in valores_coincidentes:
        df1.loc[df1[columna_a] == valor, columna_c] = df2.loc[df2[columna_b] == valor, columna_d].values[0]

    return df1

In [28]:
def Filtrar_Filas(df, Columna, Valor, *Exacto):
    
    """
    Filtra las filas del DataFrame 'df' donde los valores de la columna 'Columna' contenga a un 'Valor'.
    
    Parámetros:
    - df: dataFrame sobre el cual se realizará el filtrado.
    - Columna: nombre de la columna que se usará para la comparación.
    - Valor: valor que se buscará en la columna especificada.
    - Exacto: indica si la búsqueda debe ser exacta o no. Por defecto, es False.
    
    Retorna:
    - df con las filas filtradas.
    
    """
    
    if Exacto:
        return df[df[Columna] == Valor]
    else:
        return df[df[Columna].str.contains(Valor)]

In [29]:
def Sumar_Filas_de_Dataframes(*args):
    
    """
    Suma las filas de varios dataframes y las concatena al final del primero.

    Parámetros:
    - args: dataframes a ser sumados.

    Retorna:
    - DataFrame resultante con las filas de todos los DataFrames sumados al final del primero.
    """
    
    # Combinar todos los DataFrames en uno solo
    Combinacion = pd.concat(args, ignore_index=True)
    return Combinacion

In [30]:
def Rellenar_Columna(df, Columna, Valor):
    
    """
    Rellena toda una columna del DataFrame con un valor específico.

    Parámetros:
    - df: df en el que se realizará el relleno.
    - columna: nombre de la columna que se rellenará.
    - valor: valor con el que se rellenará la columna.

    Retorna:
    - DataFrame con la columna rellenada con el valor especificado.
    
    """
    
    df[Columna] = Valor
    return df

In [31]:
def Filtrar_y_Rellenar(df, Columna_Filtrada, Filtro, Columna_A_Rellenar, Relleno):
    
    """
    Filtra las filas del DataFrame 'df' donde los valores de la columna Columna_Filtrada contengan Filtro,
    y luego rellena la columna Columna_A_Rellenar con el valor Relleno en esas filas.

    Parámetros:
    - df: DataFrame sobre el cual se realizará el filtrado y rellenado.
    - Columna_Filtrada: nombre de la columna que se usará para el filtrado.
    - Filtro: valor que se buscará en la columna Columna_Filtrada.
    - Columna_A_Rellenar: nombre de la columna que se rellenará.
    - Relleno: valor con el que se rellenará la columna Columna_A_Rellenar en las filas filtradas.

    Retorna:
    - DataFrame completo con las modificaciones realizadas.
    """
    
    # Filtrar las filas donde los valores de la columna Columna_Filtrada contengan Filtro
    Filas_Filtradas = Filtrar_Filas(df, Columna_Filtrada, Filtro)
    
    # Rellenamos la Columna_A_Rellenar con el Relleno 
    Filas_Filtradas = Rellenar_Columna(Filas_Filtradas, Columna_A_Rellenar, Relleno)
    
    # Eliminar las filas filtradas del DataFrame original
    df = df.drop(Filas_Filtradas.index)
    
    df = Sumar_Filas_de_Dataframes(df, Filas_Filtradas)
    
    return df

In [32]:
def Aumento_Porcentual(df, Columna_Filtrada, Filtro, Columna_Operada_A, Columna_Operada_B, Porcentaje): 
    
    """
    Filtra las filas del DataFrame 'df' donde los valores de la columna Columna_Filtrada contengan Filtro,
    y luego aplica un aumento de valor Porcentaje a Columna_Operada en esas filas.

    Parámetros:
    - df: DataFrame sobre el cual se realizará el filtrado y rellenado.
    - Columna_Filtrada: nombre de la columna que se usará para el filtrado.
    - Filtro: valor que se buscará en la columna Columna_Filtrada.
    - Columna_Operada_A: nombre de la primera columna que se va a aumentar.
    - Columna_Operada_B: nombre de la segunda columna que se va a aumentar.
    - Porcentaje: valor de aumento para los valores de la Columna_Operada.

    Retorna:
    - DataFrame completo con las modificaciones realizadas.
    """
    
    # Filtrar las filas donde los valores de la columna Columna_Filtrada contengan Filtro
    Filas_Filtradas = Filtrar_Filas(df, Columna_Filtrada, Filtro)
    
    # Eliminar las filas filtradas del DataFrame original
    df = df.drop(Filas_Filtradas.index)
    
    # Aplicamos el aumento porcentual a la columna
    Filas_Filtradas[Columna_Operada_A] *= (1 + (Porcentaje / 100))
    Filas_Filtradas[Columna_Operada_B] *= (1 + (Porcentaje / 100))
    
    df = Sumar_Filas_de_Dataframes(df, Filas_Filtradas)
    
    return df

In [33]:
def Redondear_Arriba_Multiplo(df, Columna, Numero):
    
    """
    Redondea hacia arriba los valores de la columna especificada del DataFrame al múltiplo más cercano de Numero.

    Parámetros:
    - df: DataFrame sobre el cual se realizará el redondeo.
    - Columna: Nombre de la columna que se redondeará.
    - Numero: Valor numérico al cual se redondearán los valores de la columna.

    Retorna:
    - DataFrame con la columna especificada redondeada hacia arriba al múltiplo más cercano de Numero.
    """
    
    # Redondear hacia arriba al múltiplo más cercano de valor_b
    df[Columna] = np.ceil(df[Columna] / Numero) * Numero
    
    return df

In [34]:
def Guarda_Excel_Diseñado(df, Nombre):
    
    """
    Guarda un DataFrame en un archivo Excel con el nombre y ubicación especificados.
    """
    
    # Crear un nuevo libro de trabajo de Excel
    wb = Workbook()
    ws = wb.active
    
    # Definir el estilo de borde
    thin_border = Border(left=Side(style='thin'), 
                         right=Side(style='thin'), 
                         top=Side(style='thin'), 
                         bottom=Side(style='thin'))

    # Agregar los datos del DataFrame al archivo Excel
    for r in dataframe_to_rows(df, index=False, header=True):
        ws.append(r)

    # Centrar el contenido de todas las celdas
    for row in ws.iter_rows(min_row=1, min_col=1, max_col=len(df.columns), max_row=len(df)+1):
        for cell in row:
            cell.alignment = Alignment(horizontal='center', vertical='center')

    # Ajustar el ancho de las columnas automáticamente
    for col in ws.columns:
        max_length = 0
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2) * 1.2
        ws.column_dimensions[cell.column_letter].width = adjusted_width

    # Formatear las columnas específicas
    for col_num in [3, 4, 7, 9, 10, 11]:
        for row_num in range(2, len(df)+2):
            cell = ws.cell(row=row_num, column=col_num)
            cell.number_format = '$#,##0'  # Formato de moneda sin decimales

    # Formatear la columna 5 con un decimal
    for row_num in range(2, len(df)+2):
        cell = ws.cell(row=row_num, column=5)
        cell.number_format = '#,##0.0'  # Formato con un decimal       

    # Aplicar bordes y formato de color a las celdas con valores no nulos
    for row_num in range(1, len(df)+2):
        for col_num in range(1, len(df.columns)+1):
            cell = ws.cell(row=row_num, column=col_num)
            if cell.value is not None:
                # Aplicar bordes a las celdas con valores no nulos
                cell.border = thin_border
                # Aplicar formato de color a la primera fila con valores no nulos
                if row_num == 1:
                    cell.fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
                    cell.font = Font(bold=True)

    ruta = f"J:\\Mi unidad\\Forraje\\{Nombre}.xlsx"

    # Guardar el archivo Excel
    wb.save(ruta)

In [35]:
def Comparar_y_Eliminar_Filas(df1, df2, Col_A, Col_B):
    """
    Compara los valores de la columna A de df1 con los de la columna B de df2. 
    Elimina las filas de df1 que tienen valores coincidentes en ambas columnas.

    Args:
    df1 (DataFrame): Primer DataFrame.
    df2 (DataFrame): Segundo DataFrame.
    Col_A (str): Nombre de la columna en df1.
    Col_B (str): Nombre de la columna en df2.

    Returns:
    DataFrame: DataFrame df1 con las filas eliminadas.
    """
    # Copiar el DataFrame df1 para evitar modificar el original
    df1_Nuevo = df1.copy()

    # Obtener los valores únicos de la columna B de df2
    Valores_Col_B_df2 = set(df2[Col_B])

    # Filtrar filas de df1 donde la columna A no está en los valores de la columna B de df2
    df1_Nuevo = df1_Nuevo[~df1_Nuevo[Col_A].isin(Valores_Col_B_df2)]

    return df1_Nuevo